In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np

# Solution for Question 1

In [3]:
# All files have been formatted to remove undesired records from top & saved as csv
hdfc_file = 'Q1-Data/HDFC Nifty ETF.csv'
kotak_file = 'Q1-Data/KOTAK Nifty ETF.csv'
reliance_file = 'Q1-Data/Reliance Nifty ETF.csv'
uti_file = 'Q1-Data/UTI Nifty ETF.csv'

nifty_file = 'Q1-Data/NIFTY-TotalReturnsIndex.csv'

In [4]:
# Read ETF data files
# Convert string data with commas to float
# Rename 'NAV' column to 'Close'
hdfc = pd.read_csv(hdfc_file, index_col=0, parse_dates=True, thousands=',', dtype=float)
kotak = pd.read_csv(kotak_file, index_col=0, parse_dates=True, header=0, names=['Date', 'Close'], thousands=',', dtype=float)
reliance = pd.read_csv(reliance_file, index_col=0, parse_dates=True, header=0, names=['Date', 'Close'], thousands=',', dtype=float)
uti = pd.read_csv(uti_file, index_col=0, parse_dates=True, header=0, names=['Date', 'Close'], thousands=',', dtype=float)

In [5]:
# Dictionary to store ETF data
etfs = {
    'HDFC': hdfc,
    'Kotak': kotak,
    'Reliance': reliance,
    'UTI': uti
}

In [6]:
# Display first and last ETF close prices
for etf, val in etfs.items():
    print(f"""ETF Name: {etf}, Records: {val.shape[0]}, 
    Close Price -> {str(val.index[0])[:10]}: {val.iloc[0].Close}, {str(val.index[-1])[:10]}: {val.iloc[-1].Close}""")

ETF Name: HDFC, Records: 494, 
    Close Price -> 2016-01-01: 790.5, 2017-12-29: 1071.0
ETF Name: Kotak, Records: 495, 
    Close Price -> 2016-01-01: 80.51, 2017-12-29: 106.28
ETF Name: Reliance, Records: 495, 
    Close Price -> 2016-01-01: 805.06, 2017-12-29: 1084.48
ETF Name: UTI, Records: 468, 
    Close Price -> 2016-01-01: 794.05, 2017-12-29: 1090.9


In [7]:
# Read Nifty returns files
# Rename 'Total Returns Index' column to 'Close'
nifty = pd.read_csv(nifty_file, index_col=0, parse_dates=True, header=0, names=['Date', 'Close'])

In [8]:
# Display first and last benchmark close prices
print(f"""NIFTY, Records: {val.shape[0]}, 
    Close Price -> {str(val.index[0])[:10]}: {val.iloc[0].Close}, {str(val.index[-1])[:10]}: {val.iloc[-1].Close}""")

NIFTY, Records: 468, 
    Close Price -> 2016-01-01: 794.05, 2017-12-29: 1090.9


In [9]:
def tracking_error(benchmark, etf, year=None):
    """Calculates annualized tracking error of ETF funds
    @params:
    benchmark = pandas dataframe with benchmark daily close ('Close' column) indexed by date
    etf = pandas dataframe with ETF daily close indexed by date
    year = calendar year for tracking error calculation. Default None for calculating for entire period
    @returns:
    te = tracking error
    """    
    # Calculate daily returns of benchmark & ETF & store in dataframes
    benchmark_rets = benchmark.Close.pct_change().to_frame('benchmark_rets')
    etf_rets = etf.Close.pct_change().to_frame('etf_rets')
    
    # Merge benchmark & etf returns by date & fill missing returns with previous day's returns
    rets = benchmark_rets.merge(etf_rets, how='outer', on='Date')
    rets.sort_index(inplace=True)
    rets.fillna(method='ffill', inplace=True)
    rets.dropna(inplace=True) # Drop first record with nan value in both columns
    
    # Filter by year if year is available
    if year:
        rets = rets.loc[str(year)]
    
    # Calculate annualized tracking error
    N = rets.shape[0]
    te_d = np.sqrt(np.sum(np.square(rets.benchmark_rets - rets.etf_rets)) / (N - 1)) # daily tracking error
    te = np.sqrt(252) * te_d # annualized tracking error
    
    return te

In [10]:
te = pd.DataFrame(columns=['2016', '2017']) # Create empty dataframe to store tracking errors
for etf, val in etfs.items(): # Calculate & store annualized tracking errors for all ETFs
    te_2016 = tracking_error(nifty, val, '2016')
    te_2017 = tracking_error(nifty, val, '2017')
    # Append ETF TE in dataframe
    te.loc[etf] = [te_2016, te_2017]
te

,2016,2017
HDFC,0.115929,0.059478
Kotak,0.052118,0.038773
Reliance,0.037863,0.023023
UTI,0.115612,0.087238


In [11]:
# Arrange in ascending order of 2016 annualized tracking error
te['2016'].sort_values()

Reliance    0.037863
Kotak       0.052118
UTI         0.115612
HDFC        0.115929
Name: 2016, dtype: float64

In [12]:
# Arrange in ascending order of 2017 annualized tracking error
te['2017'].sort_values()

Reliance    0.023023
Kotak       0.038773
HDFC        0.059478
UTI         0.087238
Name: 2017, dtype: float64

In [13]:
# Funds with increase in annualized TE from 2016 to 2017
te[te['2016'] < te['2017']] # No fund

,2016,2017


In [14]:
# Funds with decrease in annualized TE from 2016 to 2017
te[te['2016'] > te['2017']] # All four funds

,2016,2017
HDFC,0.115929,0.059478
Kotak,0.052118,0.038773
Reliance,0.037863,0.023023
UTI,0.115612,0.087238


# Solution for Question 2

In [15]:
from functools import reduce

In [16]:
# All files have been formatted to remove undesired records from top & saved as csv
nifty_etf_file = 'Q2-Data/Nifty ETF.csv'
gold_etf_file = 'Q2-Data/Gold ETF.csv'
junior_etf_file = 'Q2-Data/Junior ETF.csv'

In [17]:
# Read ETF data files
# Convert string data with commas to float
nifty = pd.read_csv(nifty_file, index_col=0, parse_dates=True, header=0, names=['Date','nifty'], thousands=',', dtype=float)
gold = pd.read_csv(gold_etf_file, index_col=0, parse_dates=True, header=0, names=['Date', 'gold'], thousands=',', dtype=float)
jr = pd.read_csv(junior_etf_file, index_col=0, parse_dates=True, header=0, names=['Date', 'junior'], thousands=',', dtype=float)

In [23]:
# Merge all dataframes by date & fill missing values with previous day's value
etfs = reduce(lambda left,right: pd.merge(left, right, how='outer', on='Date'), [nifty, gold, jr])
etfs.sort_index(inplace=True)
etfs.fillna(method='ffill', inplace=True)
etfs
etfs.shape

,nifty,gold,junior
Date,,,
2016-01-01,10598.00,2280.20,203.62
2016-01-04,10369.24,2304.70,200.58
2016-01-05,10360.42,2319.65,201.32
2016-01-06,10302.32,2333.10,199.98
2016-01-07,10072.45,2366.80,195.78
...,...,...,...
2017-12-22,14330.49,2579.45,313.16
2017-12-26,14383.04,2600.30,314.64
2017-12-27,14327.42,2616.55,313.20


(495, 3)

In [39]:
# Calculate ETF daily returns
etf_rets = etfs.copy()
etf_rets['nifty'] = 1 + etf_rets['nifty'].pct_change()
etf_rets['gold'] = 1 + etf_rets['gold'].pct_change()
etf_rets['junior'] = 1 + etf_rets['junior'].pct_change()
etf_rets.fillna(1, inplace=True)
etf_rets

,nifty,gold,junior
Date,,,
2016-01-01,1.000000,1.000000,1.000000
2016-01-04,0.978415,1.010745,0.985070
2016-01-05,0.999149,1.006487,1.003689
2016-01-06,0.994392,1.005798,0.993344
2016-01-07,0.977688,1.014444,0.978998
...,...,...,...
2017-12-22,1.005050,1.001553,1.005006
2017-12-26,1.003667,1.008083,1.004726
2017-12-27,0.996133,1.006249,0.995423


In [41]:
# Calculate cumulative returns
etf_rets['nifty'] = etf_rets['nifty'].cumprod()
etf_rets['gold'] = etf_rets['gold'].cumprod()
etf_rets['junior'] = etf_rets['junior'].cumprod()
etf_rets

,nifty,gold,junior
Date,,,
2016-01-01,1.000000,1.000000,1.000000
2016-01-04,0.978415,1.010745,0.985070
2016-01-05,0.977583,1.017301,0.988704
2016-01-06,0.972100,1.023200,0.982124
2016-01-07,0.950410,1.037979,0.961497
...,...,...,...
2017-12-22,1.352188,1.131238,1.537963
2017-12-26,1.357147,1.140382,1.545231
2017-12-27,1.351898,1.147509,1.538159


In [46]:
etf_rets.loc['2016-03-25':'2016-04-02']

,nifty,gold,junior
Date,,,
2016-03-28,0.958639,1.111196,0.906787
2016-03-29,0.956359,1.115297,0.905707
2016-03-30,0.973882,1.126853,0.923632
2016-03-31,0.974288,1.124353,0.923976
2016-04-01,0.971097,1.130975,0.930901


In [47]:
etfs.loc['2016-03-25':'2016-04-02']

,nifty,gold,junior
Date,,,
2016-03-28,10159.66,2533.75,184.64
2016-03-29,10135.49,2543.10,184.42
2016-03-30,10321.20,2569.45,188.07
2016-03-31,10325.50,2563.75,188.14
2016-04-01,10291.69,2578.85,189.55


In [51]:
etf_rets.resample('1Q', convention='end').asfreq()

,nifty,gold,junior
Date,,,
2016-03-31,0.974288,1.124353,0.923976
2016-06-30,1.049386,1.218007,1.011345
2016-09-30,1.094378,1.239014,1.129948
2016-12-31,NaN,NaN,NaN
2017-03-31,1.170663,1.145886,1.243051
2017-06-30,1.220261,1.130120,1.309449
2017-09-30,NaN,NaN,NaN
2017-12-31,NaN,NaN,NaN


In [54]:
quarter = pd.PeriodIndex(etf_rets.index, freq='Q', name='Quarter').to_timestamp()
result = etf_rets.groupby([quarter]).last()
result

,nifty,gold,junior
Quarter,,,
2016-01-01,0.974288,1.124353,0.923976
2016-04-01,1.049386,1.218007,1.011345
2016-07-01,1.094378,1.239014,1.129948
2016-10-01,1.041745,1.124726,1.061585
2017-01-01,1.170663,1.145886,1.243051
2017-04-01,1.220261,1.130120,1.309449
2017-07-01,1.259859,1.169305,1.370838
2017-10-01,1.357041,1.152004,1.543611


In [223]:
def invest(funds, prices, ratio):
    """Invests funds into instruments with prices as per ratio
    @params:
    funds = total money to be invested
    prices = list of prices of instruments
    ratio = desired investment ratio
    @returns:
    cash, units = uninvested cash after investment, list of units in each instrument after investment
    """
    assert len(prices) == len(ratio), f'prices count {len(prices)} != ratio count {len(ratio)}'
    
    # Allocate funds to a separate pot for each instrument as per ratio
    fund_pots = [r / sum(ratio) * funds for r in ratio]
    
    units = [int(f // p) for f, p in zip(fund_pots, prices)]
    invested = [p * u for p, u in zip(prices, units)]
    cash = funds - sum(invested)
    
    ratio = [np.round(i / sum(invested), 2) for i in invested]
    print(f'Portfolio Cash: {np.round(cash, 2)} ## Portfolio value: {np.round(sum(invested, cash), 2)} ## Investments ratio: {ratio} ## Units: {units}')
    
    return cash, units

In [117]:
def rebalance(prices, units, ratio, cash):
    """Rebalances cash + investments as per ratio
    @params:
    prices = list of prices of instruments
    units = list of units in each instrument
    ratio = desired rebalance ratio
    cash = uninvested cash in fund
    @returns:
    cash, units = uninvested cash after rebalance, list of units in each instrument after rebalance
    """
    assert len(prices) == len(units) == len(ratio), \
        f'prices count {len(prices)} != units count {len(units)} != ratio count {len(ratio)}'
    
    total_investments = sum([p * u for p, u in zip(prices, units)])
    total_val = cash + total_investments
    
    cash, units = invest(total_val, prices, ratio) 
    return cash, units

In [281]:
def redeem(prices, units, cash):
    """Portfolio redemption: go all cash
    @params:
    prices = list of prices of instruments
    units = list of units in each instrument
    cash = end portfolio amount
    @returns:
    cash, units = uninvested cash after redemption, list of units in each instrument after rebalance
    """
    assert len(prices) == len(units), f'prices count {len(prices)} != units count {len(units)}'
    
    total_investments = sum([p * u for p, u in zip(prices, units)])
    cash = cash + total_investments
    total_investments = 0
    units = [0 for u in units]
    print(f'Portfolio Cash: {np.round(cash, 2)} ## Portfolio value: {np.round(cash + total_investments, 2)} ## Units: {units}')
    
    return cash, units

In [236]:
data = etfs.copy()
# Add quarter end indicator
data['month'] = data.index.month
data.loc[data['month'] % 3 != 0, 'month'] = 0
data['nxt_month'] = data['month'].shift(-1)
data.fillna(0, inplace=True)
data.loc[data['month'] - data['nxt_month'] > 0, 'qtr_end'] = 1
data.drop(columns=['month', 'nxt_month'], inplace = True)
# Create empty columns for allocation units
data = data.assign(nifty_units = np.nan, gold_units = np.nan, junior_units = np.nan, cash = np.nan)
data.loc[data['qtr_end'] == 1]

,nifty,gold,junior,qtr_end,nifty_units,gold_units,junior_units,cash
Date,,,,,,,,
2016-03-31,10325.50,2563.75,188.14,1.0,NaN,NaN,NaN,NaN
2016-06-30,11121.39,2777.30,205.93,1.0,NaN,NaN,NaN,NaN
2016-09-30,11598.22,2825.20,230.08,1.0,NaN,NaN,NaN,NaN
2016-12-30,11040.41,2564.60,216.16,1.0,NaN,NaN,NaN,NaN
2017-03-31,12406.69,2612.85,253.11,1.0,NaN,NaN,NaN,NaN
2017-06-30,12932.33,2576.90,266.63,1.0,NaN,NaN,NaN,NaN
2017-09-29,13351.99,2666.25,279.13,1.0,NaN,NaN,NaN,NaN
2017-12-29,14381.92,2626.80,314.31,1.0,NaN,NaN,NaN,NaN


In [237]:
from datetime import date

initial_capital = 100000000 # 100 million INR
start_date = date(2016, 1, 1) # Portfolio Allocation Start Date
end_date = date(2017, 12, 29) # Portfolio Redemption Date
allocation_ratio = [5, 2, 3] # Allocation ratio of 5:2:3
prev_units = [0, 0, 0] # Unit allocation initialization
prev_cash = 0 # Cash initialization

print('Start....')
print('Investments ratio & Units in order of [Nifty, Gold, Junior]')
print('===========================================================')

for row in data.itertuples():
    date, nifty, gold, junior, qtr_end = row[0], row[1], row[2], row[3], row[4]
    if date ==  start_date:
        # Portfolio allocation begin
        print(date)
        print(f'Initial Capital : {initial_capital}')
        print('Begin portfolio allocation')
        cash, units = invest(initial_capital, [nifty, gold, junior], allocation_ratio) # Initial allocation
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], data.loc[date]['junior_units'] = units # Units assignment
        data.loc[date]['cash'] = cash
        prev_units, prev_cash = units, cash
        print('===========================================================')
    elif date == end_date:
        # Portfolio redemption
        print('Portfolio redemption')
        cash, units = redeem()
        print('===========================================================')
    elif qtr_end == 1 and date != end_date:
        # Quarter end rebalancing
        print(date)
        print(f'Fund value : {initial_capital}')
        print('Begin portfolio rebalance')
        cash, units = rebalance([nifty, gold, junior], prev_units, allocation_ratio, cash) # Rebalance
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], data.loc[date]['junior_units'] = units # Units assignment
        data.loc[date]['cash'] = cash
        prev_units, prev_cash = units, cash
        print('===========================================================')
    else:
        # No change in units assignment & cash
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], \
            data.loc[date]['junior_units'] = prev_units
        data.loc[date]['cash'] = prev_cash

Start....
Investments ratio & Units in order of [Nifty, Gold, Junior]
2016-01-01 00:00:00
Initial Capital : 100000000
Begin portfolio allocation
Portfolio Cash: 9654.34 ## Portfolio value: 100000000.0 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4717, 8771, 147333]
2016-03-31 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 3616.81 ## Portfolio value: 98920919.71 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4790, 7716, 157735]
2016-06-30 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 13062.34 ## Portfolio value: 107187090.26 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4818, 7718, 156150]
2016-09-30 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 6504.34 ## Portfolio value: 113625171.9 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4898, 8043, 148155]
2016-12-30 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 11014.67 ## Portfolio value: 106734695.12 ## Investments rati

NameError: name 'redeem' is not defined

In [246]:
data = etfs.copy()
# Create empty columns for allocation units
data = data.assign(nifty_units = np.nan, gold_units = np.nan, junior_units = np.nan, cash = np.nan)

In [249]:
months = data.index.month
months

Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
            ...
            12, 12, 12, 12, 12, 12, 12, 12, 12, 12],
           dtype='int64', name='Date', length=495)

In [282]:
from datetime import date as _date

initial_capital = 100000000 # 100 million INR
start_date = _date(2016, 1, 1) # Portfolio Allocation Start Date
end_date = _date(2017, 12, 29) # Portfolio Redemption Date
allocation_ratio = [5, 2, 3] # Allocation ratio of 5:2:3
prev_units = [0, 0, 0] # Unit allocation initialization
prev_cash = 0 # Cash initialization
month = data.index.month

print('Start....')
print('Investments ratio & Units in order of [Nifty, Gold, Junior]')
print('===========================================================')

for i in range(0, data.shape[0]): # Loop through all trading days
    date, nifty, gold, junior = pd.to_datetime(data.index.values[i]), data.iloc[i]['nifty'], data.iloc[i]['gold'], data.iloc[i]['junior']
    # Identify quarter end dates for rebalancing
    qtr_end = False
    if date != end_date:
        qtr_end = True if month[i] != month[i + 1] and month[i] in (3, 6, 9, 12) else False
    
    if date ==  start_date:
        # Portfolio allocation begin
        print(date)
        print(f'Initial Capital : {initial_capital}')
        print('Begin portfolio allocation')
        cash, units = invest(initial_capital, [nifty, gold, junior], allocation_ratio) # Initial allocation
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], data.loc[date]['junior_units'] = units # Units assignment
        data.loc[date]['cash'] = cash
        prev_units, prev_cash = units, cash
        print('===========================================================')
    elif date == end_date:
        # Portfolio redemption
        print('Portfolio redemption')
        cash, units = redeem([nifty, gold, junior], prev_units,cash) # Redeem portfolio, go all cash
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], data.loc[date]['junior_units'] = units # Units assignment
        data.loc[date]['cash'] = cash
        print('===========================================================')
    elif qtr_end == True:
        # Quarter end rebalancing
        print(date)
        print(f'Fund value : {initial_capital}')
        print('Begin portfolio rebalance')
        cash, units = rebalance([nifty, gold, junior], prev_units, allocation_ratio, cash) # Rebalance
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], data.loc[date]['junior_units'] = units # Units assignment
        data.loc[date]['cash'] = cash
        prev_units, prev_cash = units, cash
        print('===========================================================')
    else:
        # No change in units assignment & cash
        data.loc[date]['nifty_units'], data.loc[date]['gold_units'], \
            data.loc[date]['junior_units'] = prev_units
        data.loc[date]['cash'] = prev_cash

Start....
Investments ratio & Units in order of [Nifty, Gold, Junior]
2016-01-01 00:00:00
Initial Capital : 100000000
Begin portfolio allocation
Portfolio Cash: 9654.34 ## Portfolio value: 100000000.0 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4717, 8771, 147333]
2016-03-31 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 3616.81 ## Portfolio value: 98920919.71 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4790, 7716, 157735]
2016-06-30 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 13062.34 ## Portfolio value: 107187090.26 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4818, 7718, 156150]
2016-09-30 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 6504.34 ## Portfolio value: 113625171.9 ## Investments ratio: [0.5, 0.2, 0.3] ## Units: [4898, 8043, 148155]
2016-12-30 00:00:00
Fund value : 100000000
Begin portfolio rebalance
Portfolio Cash: 11014.67 ## Portfolio value: 106734695.12 ## Investments rati